In [1]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing as fch

housing = fch()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
labels = housing.target.reshape(-1, 1)

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
XT = tf.transpose(X)
# this is the normal equation
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sesh:
    theta_value = theta.eval()
    
theta_value

C:\Users\tdele\Anaconda3\envs\ml\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


array([[-3.7465141e+01],
       [ 4.3573415e-01],
       [ 9.3382923e-03],
       [-1.0662201e-01],
       [ 6.4410698e-01],
       [-4.2513184e-06],
       [-3.7732250e-03],
       [-4.2664889e-01],
       [-4.4051403e-01]], dtype=float32)

In [2]:
from sklearn.preprocessing import StandardScaler

scaled_housing_plus_bias = StandardScaler().fit_transform(housing_data_plus_bias)

In [3]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * batch_index + batch_size)
    batch_start = batch_index*batch_size
    batch_end = min((batch_index+1)*batch_size, m)
    X_batch = scaled_housing_plus_bias[batch_start : batch_end]
    y_batch = labels[batch_start : batch_end]
    return X_batch, y_batch

In [4]:
from datetime import datetime
now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
root_logdir = 'tf_logs'
logdir = r'./{}/run-{}/'.format(root_logdir, now)

n_epochs = 1000
learning_rate = 0.01
batch_size = 500
n_batches = int(np.ceil(m / batch_size)) # Compute number of batches

# Changing from tf.constant to tf.placeholder
# X = tf.constant(scaled_housing_plus_bias, dtype=tf.float32, name='X')
# y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')

tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None, n + 1), name='X')
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')

theta = tf.Variable(tf.random_uniform([n + 1, 1],
                                      -1.0,
                                      1.0),
                    dtype=tf.float32,
                   name='theta')

y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
## Option 1: The Manual Way
# gradients = 2/m * tf.matmul(tf.transpose(X), error)
# training_op = tf.assign(theta, theta - learning_rate * gradients)

## Option 2: Slightly less manual with tf.gradients, but still have to define the training_op
# gradients = tf.gradients(mse, [theta])[0]
# training_op = tf.assign(theta, theta - learning_rate * gradients)

## Option 3: Highly streamlined while still being able to change things at the level of each line
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sesh:
    sesh.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sesh.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if epoch % 100 == 0:
            print("saving model checkpoint...")
            save_path = saver.save(sesh, r'./tmp/my_model.ckpt')
    best_theta = theta.eval()
    save_path = saver.save(sesh, r'/tmp/final_model.ckpt')
    file_writer.close()

saving model checkpoint...
saving model checkpoint...
saving model checkpoint...
saving model checkpoint...
saving model checkpoint...
saving model checkpoint...
saving model checkpoint...
saving model checkpoint...
saving model checkpoint...
saving model checkpoint...


In [26]:
best_theta

array([[ 0.5908909 ],
       [ 0.9250404 ],
       [ 0.12808526],
       [-0.37794524],
       [ 0.40590948],
       [ 0.02456381],
       [-0.03437319],
       [-0.6100265 ],
       [-0.95141894]], dtype=float32)